In [1]:
from pyspark.sql import SparkSession

# Add here your team number teamx
team = "team5"

# location of your Hive database in HDFS
warehouse = "project/hive/warehouse"

spark = SparkSession.builder\
        .appName("{} - spark ML".format(team))\
        .master("yarn")\
        .config("hive.metastore.uris", "thrift://hadoop-02.uni.innopolis.ru:9883")\
        .config("spark.sql.warehouse.dir", warehouse)\
        .config("spark.sql.avro.compression.codec", "snappy")\
        .enableHiveSupport()\
        .getOrCreate()

#We can also add
# .config("spark.sql.catalogImplementation","hive")\ 
# But this is the default configuration
# You can switch to Spark Catalog by setting "in-memory" for "spark.sql.catalogImplementation"

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/18 23:31:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/18 23:31:07 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
25/05/18 23:31:07 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
spark.sql("SHOW DATABASES").show()
spark.sql("USE team5_projectdb").show()
spark.sql("SHOW TABLES").show()
# spark.sql("SELECT * FROM <db_name>.<table_name>").show()

+--------------------+
|           namespace|
+--------------------+
|             default|
|             retake1|
|             root_db|
|                show|
|     team0_projectdb|
|    team11_projectdb|
|           team12_db|
|team12_hive_proje...|
|    team12_projectdb|
|    team13_projectdb|
|    team14_projectdb|
|    team15_projectdb|
|    team16_projectdb|
|    team17_projectdb|
|    team18_projectdb|
|    team19_projectdb|
|     team1_projectdb|
|    team20_projectdb|
|    team21_projectdb|
| team21_projectdb_v2|
+--------------------+
only showing top 20 rows

++
||
++
++

+---------------+--------------+-----------+
|      namespace|     tableName|isTemporary|
+---------------+--------------+-----------+
|team5_projectdb|accidents_part|      false|
|team5_projectdb|locations_part|      false|
|team5_projectdb|    q1_results|      false|
|team5_projectdb|    q2_results|      false|
|team5_projectdb|    q3_results|      false|
|team5_projectdb|    q4_results|      false|
|tea

In [3]:
tables = [row['tableName'] for row in spark.sql("SHOW TABLES").select('tableName').collect()]
tables = [table for table in tables if "_results" not in table]

for table in tables:
    print(table)
    spark.sql(f"SHOW COLUMNS IN team5_projectdb.{table}").show()

accidents_part
+------------+
|    col_name|
+------------+
|          id|
|      source|
| distance_mi|
| description|
| location_id|
|  weather_id|
| twilight_id|
|road_feat_id|
|  start_time|
|    end_time|
|       month|
|    severity|
|        year|
+------------+

locations_part
+---------+
| col_name|
+---------+
|       id|
|start_lat|
|start_lng|
|  end_lat|
|  end_lng|
|   street|
|     city|
|   county|
|  zipcode|
| timezone|
|    state|
+---------+

road_features
+---------------+
|       col_name|
+---------------+
|             id|
|        amenity|
|           bump|
|       crossing|
|       give_way|
|       junction|
|        no_exit|
|        railway|
|     roundabout|
|        station|
|           stop|
|traffic_calming|
| traffic_signal|
+---------------+

twilight
+--------------------+
|            col_name|
+--------------------+
|                  id|
|      sunrise_sunset|
|      civil_twilight|
|   nautical_twilight|
|astronomical_twil...|
+------------------

In [4]:
for table in tables:
    print(table)
    spark.sql(f"SELECT * FROM team5_projectdb.{table}").show(1)

accidents_part


25/05/18 23:31:31 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+---------+-------+-----------+--------------------+-----------+----------+-----------+------------+-------------------+-------------------+-----+--------+----+
|       id| source|distance_mi|         description|location_id|weather_id|twilight_id|road_feat_id|         start_time|           end_time|month|severity|year|
+---------+-------+-----------+--------------------+-----------+----------+-----------+------------+-------------------+-------------------+-----+--------+----+
|A-6710055|Source1|       1.01|Closed at Warrens...|    3764532|   3162384|          5|         258|2020-07-21 23:17:37|2020-07-21 23:47:37|    7|       4|2020|
+---------+-------+-----------+--------------------+-----------+----------+-----------+------------+-------------------+-------------------+-----+--------+----+
only showing top 1 row

locations_part
+-------+---------+---------+-------+-------+------+-------+------+----------+--------+-----+
|     id|start_lat|start_lng|end_lat|end_lng|street|   city|co

+---+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+
| id|amenity| bump|crossing|give_way|junction|no_exit|railway|roundabout|station| stop|traffic_calming|traffic_signal|
+---+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+
|  1|   true|false|    true|    true|    true|  false|  false|     false|   true|false|          false|          true|
+---+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+
only showing top 1 row

twilight
+---+--------------+--------------+-----------------+---------------------+
| id|sunrise_sunset|civil_twilight|nautical_twilight|astronomical_twilight|
+---+--------------+--------------+-----------------+---------------------+
|  1|         Night|           Day|              Day|                Night|
+---+--------------+--------------+-----------------+---------------

In [5]:
data = {}
for table in tables:
    data[table] = spark.read.format("avro").table(f"team5_projectdb.{table}")

In [6]:
data.keys()

dict_keys(['accidents_part', 'locations_part', 'road_features', 'twilight', 'weather_buck'])

In [7]:
from pyspark.sql.functions import col, count, when, isnull

# Calculate missing value percentages
total_count = data['locations_part'].count()

missing_stats = data['locations_part'].agg(
    (count(when(isnull(col("start_lat")), col("id"))) / total_count).alias("start_lat_missing"),
    (count(when(isnull(col("start_lng")), col("id"))) / total_count).alias("start_lng_missing"),
    (count(when(isnull(col("end_lat")), col("id"))) / total_count).alias("end_lat_missing"),
    (count(when(isnull(col("end_lng")), col("id"))) / total_count).alias("end_lng_missing")
)

missing_stats.show()

+-----------------+-----------------+------------------+------------------+
|start_lat_missing|start_lng_missing|   end_lat_missing|   end_lng_missing|
+-----------------+-----------------+------------------+------------------+
|              0.0|              0.0|0.3335100786915551|0.3335100786915551|
+-----------------+-----------------+------------------+------------------+



From the table we can see that we always have starting coordinates of the accident and about 33.35% of the times we don't have ending coordinates. We will impute missing coordinates with starting coordinates, as it's most likely that those accidents are single point, i.e., accident did not involve movement.

In [8]:
from pyspark.sql.functions import coalesce, col

# Impute missing end locations with start locations
data['locations_part'] = data['locations_part'].withColumn(
    "end_lat", coalesce(col("end_lat"), col("start_lat")))
    
data['locations_part'] = data['locations_part'].withColumn(
    "end_lng", coalesce(col("end_lng"), col("start_lng")))

# Add flag indicating imputation
data['locations_part'] = data['locations_part'].withColumn(
    "end_loc_imputed", 
    (col("end_lat") == col("start_lat")) & (col("end_lng") == col("start_lng")))

In [9]:
data['locations_part'].show(5)

+-------+---------+---------+-------+-------+------------------+----+----------+----------+--------+-----+---------------+
|     id|start_lat|start_lng|end_lat|end_lng|            street|city|    county|   zipcode|timezone|state|end_loc_imputed|
+-------+---------+---------+-------+-------+------------------+----+----------+----------+--------+-----+---------------+
|3797705|       39|      -76|     39|    -76|     Canterbury Rd|Kent|     19943|US/Eastern|  Felton|   DE|           true|
|3791535|       39|      -76|     39|    -76|      S Dupont Hwy|Kent|     19943|US/Eastern|  Felton|   DE|           true|
|3788821|       39|      -76|     39|    -76|      S Dupont Hwy|Kent|     19901|US/Eastern|   Dover|   DE|           true|
|3786284|       39|      -76|     39|    -76| Wheatleys Pond Rd|Kent|19977-3799|US/Eastern|  Smyrna|   DE|           true|
|3775460|       39|      -76|     39|    -76|      S Dupont Hwy|Kent|     19901|US/Eastern|   Dover|   DE|           true|
+-------+-------

In [10]:
from pyspark.sql.functions import from_unixtime, unix_timestamp, col

# Define the desired format (optional for from_unixtime, default is 'yyyy-MM-dd HH:mm:ss')
datetime_format = "yyyy-MM-dd HH:mm:ss"

data['accidents_part'] = data['accidents_part'].withColumn(
    "start_datetime",
    from_unixtime(unix_timestamp(col("start_time")), datetime_format)
).withColumn(
    "end_datetime",
    from_unixtime(unix_timestamp(col("end_time")), datetime_format)
)

data['weather_buck'] = data['weather_buck'].withColumn(
    "weather_datetime",
    from_unixtime(unix_timestamp(col("weather_timestamp")), datetime_format)
)

In [11]:
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second

# Extract components for accidents
data['accidents_part'] = data['accidents_part'].withColumn("start_year", year("start_datetime")) \
    .withColumn("start_month", month("start_datetime")) \
    .withColumn("start_day", dayofmonth("start_datetime")) \
    .withColumn("start_hour", hour("start_datetime")) \
    .withColumn("start_minute", minute("start_datetime")) \
    .withColumn("start_second", second("start_datetime"))
data['accidents_part'] = data['accidents_part'].withColumn("end_year", year("end_datetime")) \
    .withColumn("end_month", month("end_datetime")) \
    .withColumn("end_day", dayofmonth("end_datetime")) \
    .withColumn("end_hour", hour("end_datetime")) \
    .withColumn("end_minute", minute("end_datetime")) \
    .withColumn("end_second", second("end_datetime"))

# Extract components for weather
data['weather_buck'] = data['weather_buck'].withColumn("weather_year", year("weather_datetime")) \
    .withColumn("weather_month", month("weather_datetime")) \
    .withColumn("weather_day", dayofmonth("weather_datetime")) \
    .withColumn("weather_hour", hour("weather_datetime")) \
    .withColumn("weather_minute", minute("weather_datetime")) \
    .withColumn("weather_second", second("weather_datetime"))

In [12]:
from pyspark.sql.functions import col, datediff, unix_timestamp, date_format, when, dayofweek

# Duration of accident in minutes
data['accidents_part'] = data['accidents_part'].withColumn(
    "duration_minutes",
    (unix_timestamp(col("end_time")) - unix_timestamp(col("start_time"))) / 60.0
)

# Day of week (1=Sunday, 7=Saturday)
data['accidents_part'] = data['accidents_part'].withColumn(
    "day_of_week",
    dayofweek(col("start_datetime"))  # Returns 1 (Sunday) to 7 (Saturday)
)

# Weekend flag (Saturday=7, Sunday=1)
data['accidents_part'] = data['accidents_part'].withColumn(
    "is_weekend",
    ((col("day_of_week") == 1) | (col("day_of_week") == 7)).cast("integer")
)


# Season (could be useful for weather patterns)
data['accidents_part'] = data['accidents_part'].withColumn(
    "season",
    when((col("start_month") >= 3) & (col("start_month") <= 5), "spring")
    .when((col("start_month") >= 6) & (col("start_month") <= 8), "summer")
    .when((col("start_month") >= 9) & (col("start_month") <= 11), "fall")
    .otherwise("winter")
)

In [13]:
data

{'accidents_part': DataFrame[id: string, source: string, distance_mi: decimal(10,2), description: string, location_id: bigint, weather_id: bigint, twilight_id: int, road_feat_id: int, start_time: timestamp, end_time: timestamp, month: int, severity: smallint, year: int, start_datetime: string, end_datetime: string, start_year: int, start_month: int, start_day: int, start_hour: int, start_minute: int, start_second: int, end_year: int, end_month: int, end_day: int, end_hour: int, end_minute: int, end_second: int, duration_minutes: double, day_of_week: int, is_weekend: int, season: string],
 'locations_part': DataFrame[id: bigint, start_lat: decimal(10,0), start_lng: decimal(10,0), end_lat: decimal(10,0), end_lng: decimal(10,0), street: string, city: string, county: string, zipcode: string, timezone: string, state: string, end_loc_imputed: boolean],
 'road_features': DataFrame[id: int, amenity: boolean, bump: boolean, crossing: boolean, give_way: boolean, junction: boolean, no_exit: boole

In [14]:
from pyspark.sql.functions import col

# Join all tables
df = data['accidents_part'] \
    .join(data['locations_part'], data['accidents_part'].location_id == data['locations_part'].id, 'left') \
    .join(data['weather_buck'], data['accidents_part'].weather_id == data['weather_buck'].id, 'left') \
    .join(data['twilight'], data['accidents_part'].twilight_id == data['twilight'].id, 'left') \
    .join(data['road_features'], data['accidents_part'].road_feat_id == data['road_features'].id, 'left')

# Drop redundant ID columns
df = df.drop('location_id', 'weather_id', 'twilight_id', 'road_feat_id')

In [15]:
data['accidents_part'].select('id', 'description').show(10, truncate=False)

+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|id       |description                                                                                                                                              |
+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|A-5429427|Slow traffic on Southern State Pkwy E from Eagle Ave/Exit 18 (Southern State Pkwy) to Baldwin Rd/Grand Ave/Exit 20 (Southern State Pkwy) due to accident.|
|A-4575117|Accident from exit [284A] to I-30 E.                                                                                                                     |
|A-3936576|Incident on HIGH HOUSE RD near DAVIS DR Expect delays.                                                                                                   |
|A-3

As we can see, description is mainly giving us information about the location of the accidents - we already have that, so that information is not really useful and we can safely drop it.

In [16]:
from pyspark.sql.functions import col

# First, let's select only the needed columns from each table before joining
accidents_selected = data['accidents_part'].select(
    [c for c in data['accidents_part'].columns]
)

locations_selected = data['locations_part'].select(
    'id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
    'street', 'city', 'county', 'state', 'zipcode', 
    'timezone'
)

weather_selected = data['weather_buck'].select(
    'id', 'airport_code', 'temperature_f', 'wind_chill_f', 'humidity_pct', 
    'pressure_in', 'visibility_mi', 'wind_direction', 'wind_speed_mph', 
    'precipitation_in', 'weather_condition', 'weather_year', 'weather_month', 
    'weather_day', 'weather_hour', 'weather_minute', 'weather_second'
)

twilight_selected = data['twilight'].select(
    [c for c in data['twilight'].columns]
)

road_features_selected = data['road_features'].select(
    [c for c in data['road_features'].columns]
)

# Now perform the join with explicit column selection
df = accidents_selected \
    .join(locations_selected, accidents_selected.location_id == locations_selected.id, 'left') \
    .join(weather_selected, accidents_selected.weather_id == weather_selected.id, 'left') \
    .join(twilight_selected, accidents_selected.twilight_id == twilight_selected.id, 'left') \
    .join(road_features_selected, accidents_selected.road_feat_id == road_features_selected.id, 'left')

# Drop the redundant ID columns from joined tables
df = df.drop(
    locations_selected.id, 
    weather_selected.id, 
    twilight_selected.id, 
    road_features_selected.id,
    'location_id', 
    'weather_id', 
    'twilight_id', 
    'road_feat_id'
)

# Also drop any other redundant columns that might have been carried over
redundant_cols = [
    'start_datetime', 'end_datetime',  # We have the decomposed time features
    'weather_datetime',                # Same as above
    'description'                      # Text field not useful for ML
]

df = df.drop(*[c for c in redundant_cols if c in df.columns])

# Verify the remaining columns
print("Final columns after joining:")
df.printSchema()

Final columns after joining:
root
 |-- id: string (nullable = true)
 |-- source: string (nullable = true)
 |-- distance_mi: decimal(10,2) (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- month: integer (nullable = true)
 |-- severity: short (nullable = true)
 |-- year: integer (nullable = true)
 |-- start_year: integer (nullable = true)
 |-- start_month: integer (nullable = true)
 |-- start_day: integer (nullable = true)
 |-- start_hour: integer (nullable = true)
 |-- start_minute: integer (nullable = true)
 |-- start_second: integer (nullable = true)
 |-- end_year: integer (nullable = true)
 |-- end_month: integer (nullable = true)
 |-- end_day: integer (nullable = true)
 |-- end_hour: integer (nullable = true)
 |-- end_minute: integer (nullable = true)
 |-- end_second: integer (nullable = true)
 |-- duration_minutes: double (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- is_weekend: integer (nullable = t

In [17]:
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
import math

class CyclicalTimeTransformer(Transformer, HasInputCol, HasOutputCol, 
                            DefaultParamsReadable, DefaultParamsWritable):
    """
    A custom transformer that converts cyclical time features (like hour, month, etc.)
    into sin/cos components to preserve their cyclical nature.
    """
    
    period = Param(Params._dummy(), "period", "The period of the cyclical feature (e.g., 12 for months, 24 for hours)",
                  typeConverter=TypeConverters.toFloat)
    
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, period=None):
        """
        Initialize the transformer.
        
        :param inputCol: The name of the input column (time component to transform)
        :param outputCol: The base name for output columns (will append _sin and _cos)
        :param period: The period of the cyclical feature (e.g., 12 for months)
        """
        super(CyclicalTimeTransformer, self).__init__()
        self._setDefault(period=12.0)  # Default to monthly cycle
        kwargs = self._input_kwargs
        self.setParams(**kwargs)
    
    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, period=None):
        """
        Set the params for this CyclicalTimeTransformer.
        """
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def getPeriod(self):
        """
        Gets the value of period or its default value.
        """
        return self.getOrDefault(self.period)
    
    def _transform(self, dataset: DataFrame):
        """
        Transform the input dataset by adding sin/cos components of the cyclical feature.
        """
        input_col = self.getInputCol()
        output_col = self.getOutputCol()
        period = self.getPeriod()
        
        # Calculate the sin and cos components
        angle = 2 * math.pi * F.col(input_col) / period
        
        return dataset.withColumn(f"{output_col}_sin", F.sin(angle)) \
                     .withColumn(f"{output_col}_cos", F.cos(angle))

In [18]:
cols = ['start', 'end', 'weather']
cyclical_time_transformers = []
for col in cols:
    # Transform months (period=12)
    cyclical_time_transformers.append(
        CyclicalTimeTransformer(
            inputCol=f"{col}_month", 
            outputCol=f"{col}_month_encoded",
            period=12.0
        )
    )

    # Transform days (period=31)
    cyclical_time_transformers.append(
        CyclicalTimeTransformer(
            inputCol=f"{col}_day", 
            outputCol=f"{col}_day_encoded",
            period=31.0
        )
    )

    # Transform hours (period=24)
    cyclical_time_transformers.append(
        CyclicalTimeTransformer(
            inputCol=f"{col}_hour",
            outputCol=f"{col}_hour_encoded",
            period=24.0
        )
    )

    # Transform minutes (period=60)
    cyclical_time_transformers.append(
        CyclicalTimeTransformer(
            inputCol=f"{col}_minute",
            outputCol=f"{col}_minute_encoded",
            period=60.0
        )
    )

    # Transform seconds (period=60)
    cyclical_time_transformers.append(
        CyclicalTimeTransformer(
            inputCol=f"{col}_second",
            outputCol=f"{col}_second_encoded",
            period=60.0
        )
    )
len(cyclical_time_transformers) # Should be 3*5=15

15

In [19]:
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCols, HasOutputCols
import pyspark.sql.functions as F
import math

class GeodeticToECEFTransformer(Transformer, HasInputCols, HasOutputCols):
    """
    Converts geodetic coordinates (lat, lng, alt) to ECEF coordinates (x, y, z)
    WGS84 ellipsoid parameters used by default
    """
    
    @keyword_only
    def __init__(self, inputCols=None, outputCols=None):
        super(GeodeticToECEFTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)
    
    @keyword_only
    def setParams(self, inputCols=None, outputCols=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def _transform(self, dataset):
        # WGS84 parameters
        a = 6378137.0  # semi-major axis in meters
        f = 1/298.257223563  # flattening
        b = a * (1 - f)  # semi-minor axis
        
        lat_col, lng_col, alt_col = self.getInputCols()
        x_col, y_col, z_col = self.getOutputCols()
        
        # Convert to radians
        lat_rad = F.radians(F.col(lat_col))
        lng_rad = F.radians(F.col(lng_col))
        
        # Compute eccentricity
        e_sq = 2*f - f*f
        
        # Compute N (prime vertical radius of curvature)
        N = a / F.sqrt(1 - e_sq * F.sin(lat_rad)**2)
        
        # Compute ECEF coordinates
        x = (N + F.col(alt_col)) * F.cos(lat_rad) * F.cos(lng_rad)
        y = (N + F.col(alt_col)) * F.cos(lat_rad) * F.sin(lng_rad)
        z = ((1 - e_sq) * N + F.col(alt_col)) * F.sin(lat_rad)
        
        return dataset.withColumn(x_col, x) \
                     .withColumn(y_col, y) \
                     .withColumn(z_col, z)

In [20]:
geodetic_transformers = []
for pref in ['start', 'end']:
    if f'{pref}_alt' not in df.columns:
        df = df.withColumn(f'{pref}_alt', F.lit(0.0))
    geodetic_transformers.append(
        GeodeticToECEFTransformer(
            inputCols=[f'{pref}_lat', f'{pref}_lng', f'{pref}_alt'],
            outputCols=[f'{pref}_x', f'{pref}_y', f'{pref}_z']
        )
    )

len(geodetic_transformers) # Should be 2

2

In [21]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, FeatureHasher
from pyspark.sql.functions import concat_ws

# df = df.withColumn("address", concat_ws(", ", "street", "city", "county", "state", "zipcode"))

address_hasher = FeatureHasher(
    inputCols=["street", "city", "county", "state", "zipcode"],
    outputCol="address_hashed",
    numFeatures=64  # Reduced from default 2^18 to save memory
)
zipcode_indexer = StringIndexer(inputCol="zipcode", outputCol="zipcode_encoded", handleInvalid="keep")
timezone_indexer = StringIndexer(inputCol="timezone", outputCol="timezone_encoded", handleInvalid="keep")

In [22]:
for transformer in cyclical_time_transformers+geodetic_transformers:
    df = transformer.transform(df)
df = address_hasher.transform(df)
df = zipcode_indexer.fit(df).transform(df)
df = timezone_indexer.fit(df).transform(df)

In [23]:
df.head(2)

25/05/18 23:33:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(id='A-3464302', source='Source1', distance_mi=Decimal('0.50'), start_time=datetime.datetime(2016, 10, 25, 12, 15, 41), end_time=datetime.datetime(2016, 10, 25, 18, 15, 41), month=10, severity=3, year=2016, start_year=2016, start_month=10, start_day=25, start_hour=12, start_minute=15, start_second=41, end_year=2016, end_month=10, end_day=25, end_hour=18, end_minute=15, end_second=41, duration_minutes=360.0, day_of_week=3, is_weekend=0, season='fall', start_lat=Decimal('40'), start_lng=Decimal('-86'), end_lat=Decimal('40'), end_lng=Decimal('-86'), street=' N Dearborn St', city='Marion', county='46218-3744', state='IN', zipcode='US/Eastern', timezone='Indianapolis', airport_code='KEYE', temperature_f=Decimal('42'), wind_chill_f=Decimal('39'), humidity_pct=Decimal('82'), pressure_in=Decimal('30'), visibility_mi=Decimal('10'), wind_direction='East', wind_speed_mph=Decimal('5'), precipitation_in=None, weather_condition='Clear', weather_year=2016, weather_month=10, weather_day=25, weathe

In [24]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

# Select numerical features to scale
numerical_cols = [
    'distance_mi', 'duration_minutes', 'temperature_f', 'wind_chill_f',
    'humidity_pct', 'pressure_in', 'visibility_mi', 'wind_speed_mph',
    'precipitation_in', 'start_x', 'start_y', 'start_z', 'end_x', 'end_y', 'end_z'
]

# Convert string columns to double
for col_name in numerical_cols:
    if col_name in df.columns and str(df.schema[col_name].dataType) == 'StringType':
        df = df.withColumn(col_name, F.col(col_name).cast('double'))

# Remove any null values (or impute)
df = df.na.fill(0, subset=numerical_cols)

# Assemble and scale features
assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
scaler = StandardScaler(inputCol="numerical_features", outputCol="scaled_features")

# Create pipeline
pipeline = Pipeline(stages=[assembler, scaler])
scaler_model = pipeline.fit(df)
df = scaler_model.transform(df)

In [26]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# List of categorical columns
# source add later
categorical_cols = [
     'weather_condition', 'sunrise_sunset', 'civil_twilight',
    'nautical_twilight', 'astronomical_twilight', 'season','day_of_week'
]

# Add boolean road features
road_bool_cols = [c for c in df.columns if c in [
    'amenity', 'bump', 'crossing', 'give_way', 'junction', 'no_exit',
    'railway', 'roundabout', 'station', 'stop', 'traffic_calming', 'traffic_signal', 'is_weekend'
]]

categorical_cols.extend(road_bool_cols)

for col in road_bool_cols:
    df = df.withColumn(col, F.col(col).cast("string"))

# String index and one-hot encode
indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_index", handleInvalid="keep") for c in categorical_cols]
encoder = OneHotEncoder(
    inputCols=[f"{c}_index" for c in categorical_cols],
    outputCols=[f"{c}_encoded" for c in categorical_cols]
)

# Create pipeline
pipeline = Pipeline(stages=indexers + [encoder])
encoder_model = pipeline.fit(df)
df = encoder_model.transform(df)

In [27]:
from pyspark.ml.feature import VectorAssembler

# Get all feature columns
feature_cols = [
    'scaled_features',
    *[f"{c}_encoded" for c in categorical_cols],
    *[c for c in df.columns if '_encoded_sin' in c or '_encoded_cos' in c]
]

# Assemble final feature vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)

In [28]:
from pyspark.ml.feature import VarianceThresholdSelector

selector = VarianceThresholdSelector(
    varianceThreshold=0.01,
    featuresCol="features",
    outputCol="selected_features"
)

selector_model = selector.fit(df)
df = selector_model.transform(df)

# Show which features were selected
selected_features = selector_model.selectedFeatures
print(f"Selected {len(selected_features)} out of {len(feature_cols)} features")

Selected 87 out of 51 features


In [29]:
df.head(2)

[Row(id='A-3464290', source='Source1', distance_mi=Decimal('0.00'), start_time=datetime.datetime(2016, 10, 25, 12, 4, 29), end_time=datetime.datetime(2016, 10, 25, 18, 4, 29), month=10, severity=2, year=2016, start_year=2016, start_month=10, start_day=25, start_hour=12, start_minute=4, start_second=29, end_year=2016, end_month=10, end_day=25, end_hour=18, end_minute=4, end_second=29, duration_minutes=360.0, day_of_week=3, is_weekend='0', season='fall', start_lat=Decimal('40'), start_lng=Decimal('-86'), end_lat=Decimal('40'), end_lng=Decimal('-86'), street='N White River Pkwy West Dr', city='Marion', county='46222', state='IN', zipcode='US/Eastern', timezone='Indianapolis', airport_code='KEYE', temperature_f=Decimal('42'), wind_chill_f=Decimal('39'), humidity_pct=Decimal('82'), pressure_in=Decimal('30'), visibility_mi=Decimal('10'), wind_direction='East', wind_speed_mph=Decimal('5'), precipitation_in=Decimal('0'), weather_condition='Clear', weather_year=2016, weather_month=10, weather_d

In [30]:
# df is the DataFrame after VarianceThresholdSelector, containing:
# 'id', 'severity', 'duration_minutes', 'selected_features', and other original columns.

# --- Prepare data for CLASSIFICATION (Severity) ---
df_cls = df.select(
    F.col("id"),
    F.col("severity").alias("label"),  # 'severity' is the target
    F.col("selected_features").alias("features")
)
# .na.drop(subset=["label", "features"]) # Important: drop rows where label or features are null

# --- Prepare data for REGRESSION (Duration) ---
df_reg = df.select(
    F.col("id"),
    F.col("duration_minutes").alias("label"), # 'duration_minutes' is the target
    F.col("selected_features").alias("features")
)
# .na.drop(subset=["label", "features"]) # Important: drop rows where label or features are null

# Now, you will split df_classification_input for your classification models
# And you will split df_regression_input for your regression models.

In [31]:
#  split the data into 60% training and 40% test (it is not stratified)
(train_data_cls, test_data_cls) = df_cls.randomSplit([0.6, 0.4], seed = 10)
(train_data_reg, test_data_reg) = df_reg.randomSplit([0.6, 0.4], seed = 10)

In [32]:
# A function to run commands

# save to json file
import os
def run(command):
    return os.popen(command).read()

def saveDF(dataframe,name):
    dataframe.select("id", "label", "features")\
    .coalesce(1)\
    .write\
    .mode("overwrite")\
    .format("json")\
    .save(f"project/data/{name}")
    
    run(f"hdfs dfs -cat project/data/{name}/*.json > ../data/{name}.json")
    
saveDF(train_data_cls,'train_data_cls')

saveDF(test_data_cls,'test_data_cls')

saveDF(train_data_reg,'train_data_reg')

saveDF(test_data_reg,'test_data_reg')

Imoprt libs

In [34]:
from pyspark.sql.functions import col
import os
import numpy as np
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.regression import LinearRegression, GBTRegressor

Helper function to save model and predictions

In [35]:
def save_model_and_predictions(model, model_name_prefix, task_label, test_data_df, base_path="project"):
    model_full_name = f"{model_name_prefix}_{task_label}"
    
    # Save model
    model_path_hdfs = f"{base_path}/models/{model_full_name}"
    model_path_local_repo = f"../models/{model_full_name}" # Assuming 'models' folder in repo root
    
    print(f"Saving model {model_full_name} to HDFS: {model_path_hdfs}")
    model.write().overwrite().save(model_path_hdfs)
    
    # Ensure local directory exists
    os.makedirs(model_path_local_repo, exist_ok=True)
    run(f"hdfs dfs -get -f {model_path_hdfs}/* {model_path_local_repo}/") # Copy contents
    # For some models, -get needs the directory, not wildcard, and it copies the dir itself.
    # If the above fails, try: run(f"hdfs dfs -get -f {model_path_hdfs} models/") to get the folder into local 'models/'
    print(f"Model {model_full_name} copied to local repository: {model_path_local_repo}")

    # Predict
    print(f"Making predictions with {model_full_name} on test data...")
    predictions_df = model.transform(test_data_df)

    # Save predictions
    predictions_path_hdfs = f"{base_path}/output/{model_full_name}_predictions" # Full path for HDFS save
    predictions_path_local_csv_repo = f"../output/{model_full_name}_predictions.csv" # Path for final CSV in repo

    print(f"Saving predictions for {model_full_name} to HDFS: {predictions_path_hdfs}")
    predictions_df.select("label", "prediction") \
        .coalesce(1) \
        .write \
        .mode("overwrite") \
        .format("csv") \
        .option("sep", ",") \
        .option("header", "true") \
        .save(predictions_path_hdfs) # Saves as a directory with part-files

    # Ensure local output directory exists
    os.makedirs(os.path.dirname(predictions_path_local_csv_repo), exist_ok=True)
    run(f"hdfs dfs -cat {predictions_path_hdfs}/part*.csv > {predictions_path_local_csv_repo}")
    print(f"Predictions for {model_full_name} copied to local CSV: {predictions_path_local_csv_repo}")
    
    return predictions_df

TASK 1: SEVERITY PREDICTION (CLASSIFICATION)

In [36]:
train_data_cls.cache()
test_data_cls.cache()

# Evaluators for Classification
cls_evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
cls_evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

 Model 1.1: Logistic Regression for Classification

In [37]:
print("\n--- Training Classification Model 1: Logistic Regression ---")
lr_cls = LogisticRegression(featuresCol="features", labelCol="label")

lr_cls_param_grid = ParamGridBuilder() \
    .addGrid(lr_cls.regParam, [0.01, 0.1]) \
    .addGrid(lr_cls.elasticNetParam, [0.0, 0.8]) \
    .build()

lr_cls_cv = CrossValidator(estimator=lr_cls,
                           estimatorParamMaps=lr_cls_param_grid,
                           evaluator=cls_evaluator_f1, # Optimize for F1-score
                           numFolds=3, # Use 2 for faster run on large data if needed
                           parallelism=2)


--- Training Classification Model 1: Logistic Regression ---


In [38]:
print("Fitting Logistic Regression (Classification) CV model...")
lr_cls_cv_model = lr_cls_cv.fit(train_data_cls)
best_lr_cls_model = lr_cls_cv_model.bestModel
print("Best Logistic Regression (Classification) Hyperparameters:")
print(best_lr_cls_model.extractParamMap())

Fitting Logistic Regression (Classification) CV model...


25/05/19 00:17:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


Best Logistic Regression (Classification) Hyperparameters:
{Param(parent='LogisticRegression_f96ccf921211', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LogisticRegression_f96ccf921211', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_f96ccf921211', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto', Param(parent='LogisticRegression_f96ccf921211', name='featuresCol', doc='features column name.'): 'features', Param(parent='LogisticRegression_f96ccf921211', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_f96ccf921211', name='labelCol', doc='label column name.'): 'label', Param(parent='LogisticRegression_f96ccf921211', nam

In [39]:
lr_cls_predictions = save_model_and_predictions(best_lr_cls_model, "LogisticRegression", "cls", test_data_cls)

lr_cls_f1 = cls_evaluator_f1.evaluate(lr_cls_predictions)
lr_cls_accuracy = cls_evaluator_accuracy.evaluate(lr_cls_predictions)
print(f"Logistic Regression (Classification) - F1-Score on Test Data: {lr_cls_f1}")
print(f"Logistic Regression (Classification) - Accuracy on Test Data: {lr_cls_accuracy}")

Saving model LogisticRegression_cls to HDFS: project/models/LogisticRegression_cls


Model LogisticRegression_cls copied to local repository: models/LogisticRegression_cls
Making predictions with LogisticRegression_cls on test data...
Saving predictions for LogisticRegression_cls to HDFS: project/output/LogisticRegression_cls_predictions


Predictions for LogisticRegression_cls copied to local CSV: ../output/LogisticRegression_cls_predictions.csv


Logistic Regression (Classification) - F1-Score on Test Data: 0.8717594070631641
Logistic Regression (Classification) - Accuracy on Test Data: 0.9114721682003605


Model 1.2: Random Forest Classifier

In [40]:
rf_cls = RandomForestClassifier(featuresCol="features", labelCol="label", seed=42)

rf_cls_param_grid = ParamGridBuilder() \
    .addGrid(rf_cls.numTrees, [10, 20]) \
    .addGrid(rf_cls.maxDepth, [5, 10]) \
    .build()

rf_cls_cv = CrossValidator(estimator=rf_cls,
                           estimatorParamMaps=rf_cls_param_grid,
                           evaluator=cls_evaluator_f1, # Optimize for F1-score
                           numFolds=3, # Use 2 for faster run
                           parallelism=2)

Fitting Random Forest (Classification) CV model

In [41]:
print("...")
rf_cls_cv_model = rf_cls_cv.fit(train_data_cls)
best_rf_cls_model = rf_cls_cv_model.bestModel
print("Best Random Forest (Classification) Hyperparameters:")
print(best_rf_cls_model.extractParamMap())

rf_cls_predictions = save_model_and_predictions(best_rf_cls_model, "RandomForestClassifier", "cls", test_data_cls)

rf_cls_f1 = cls_evaluator_f1.evaluate(rf_cls_predictions)
rf_cls_accuracy = cls_evaluator_accuracy.evaluate(rf_cls_predictions)
print(f"Random Forest (Classification) - F1-Score on Test Data: {rf_cls_f1}")
print(f"Random Forest (Classification) - Accuracy on Test Data: {rf_cls_accuracy}")

...


25/05/19 00:55:14 WARN DAGScheduler: Broadcasting large task binary with size 1067.2 KiB
25/05/19 00:55:31 WARN DAGScheduler: Broadcasting large task binary with size 1496.7 KiB
25/05/19 00:55:52 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
25/05/19 00:56:20 WARN DAGScheduler: Broadcasting large task binary with size 1693.1 KiB
25/05/19 00:58:18 WARN DAGScheduler: Broadcasting large task binary with size 1076.8 KiB
25/05/19 00:58:34 WARN DAGScheduler: Broadcasting large task binary with size 1551.6 KiB
25/05/19 00:58:55 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
25/05/19 00:59:24 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
25/05/19 01:00:03 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
25/05/19 01:01:49 WARN DAGScheduler: Broadcasting large task binary with size 1072.7 KiB
25/05/19 01:02:07 WARN DAGScheduler: Broadcasting large task binary with size 1506.5 KiB
25/05/19 01:02:27 WARN DAGSchedul

Best Random Forest (Classification) Hyperparameters:
{Param(parent='RandomForestClassifier_98dad5c0d493', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): True, Param(parent='RandomForestClassifier_98dad5c0d493', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False, Param(parent='RandomForestClassifier_98dad5c0d493', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10, Param(parent='RandomForestClassifier_98dad5c0d493', name='featureSubsetStrategy', doc="The number of featu

25/05/19 01:15:47 WARN DAGScheduler: Broadcasting large task binary with size 1883.4 KiB


Predictions for RandomForestClassifier_cls copied to local CSV: ../output/RandomForestClassifier_cls_predictions.csv


25/05/19 01:15:53 WARN DAGScheduler: Broadcasting large task binary with size 1681.2 KiB
25/05/19 01:15:59 WARN DAGScheduler: Broadcasting large task binary with size 1681.2 KiB


Random Forest (Classification) - F1-Score on Test Data: 0.8785160419437835
Random Forest (Classification) - Accuracy on Test Data: 0.9151883123043355


Compare Classification Models

In [42]:
cls_models_summary_data = [
    (str(best_lr_cls_model), lr_cls_f1, lr_cls_accuracy),
    (str(best_rf_cls_model), rf_cls_f1, rf_cls_accuracy)
]
cls_summary_df = spark.createDataFrame(cls_models_summary_data, ["model", "F1_Score", "Accuracy"])
print("\nClassification Models Comparison:")
cls_summary_df.show(truncate=False)

cls_summary_path_hdfs = "project/output/classification_evaluation" # Directory for CSV parts
cls_summary_path_local_csv = "../output/classification_evaluation.csv"
cls_summary_df.coalesce(1) \
    .write.mode("overwrite").format("csv").option("sep", ",").option("header", "true") \
    .save(cls_summary_path_hdfs)
os.makedirs(os.path.dirname(cls_summary_path_local_csv), exist_ok=True)
run(f"hdfs dfs -cat {cls_summary_path_hdfs}/part*.csv > {cls_summary_path_local_csv}")
print(f"Classification evaluation summary saved to HDFS dir: {cls_summary_path_hdfs} and concatenated to local: {cls_summary_path_local_csv}")

train_data_cls.unpersist()
test_data_cls.unpersist()


Classification Models Comparison:


+-------------------------------------------------------------------------------------------------------------------+------------------+------------------+
|model                                                                                                              |F1_Score          |Accuracy          |
+-------------------------------------------------------------------------------------------------------------------+------------------+------------------+
|LogisticRegressionModel: uid=LogisticRegression_f96ccf921211, numClasses=5, numFeatures=87                         |0.8717594070631641|0.9114721682003605|
|RandomForestClassificationModel: uid=RandomForestClassifier_98dad5c0d493, numTrees=10, numClasses=5, numFeatures=87|0.8785160419437835|0.9151883123043355|
+-------------------------------------------------------------------------------------------------------------------+------------------+------------------+

Classification evaluation summary saved to HDFS dir: project/ou

DataFrame[id: string, label: smallint, features: vector]

TASK 2: DURATION PREDICTION (REGRESSION)

In [43]:
# Evaluators for Regression
reg_evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
reg_evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

In [44]:
# Model 2.1: Linear Regression for Regression
lr_reg = LinearRegression(featuresCol="features", labelCol="label")

lr_reg_param_grid = ParamGridBuilder() \
    .addGrid(lr_reg.regParam, [0.01, 0.1]) \
    .addGrid(lr_reg.elasticNetParam, [0.0, 0.8]) \
    .addGrid(lr_reg.aggregationDepth, [2, 3]) \
    .build()

lr_reg_cv = CrossValidator(estimator=lr_reg,
                           estimatorParamMaps=lr_reg_param_grid,
                           evaluator=reg_evaluator_rmse, # Optimize for RMSE
                           numFolds=3, # Use 2 for faster run
                           parallelism=2)

In [45]:
print("Fitting Linear Regression (Regression) CV model...")
lr_reg_cv_model = lr_reg_cv.fit(train_data_reg)
best_lr_reg_model = lr_reg_cv_model.bestModel
print("Best Linear Regression (Regression) Hyperparameters:")
print(best_lr_reg_model.extractParamMap())

lr_reg_predictions = save_model_and_predictions(best_lr_reg_model, "LinearRegression", "reg", test_data_reg)

lr_reg_rmse = reg_evaluator_rmse.evaluate(lr_reg_predictions)
lr_reg_r2 = reg_evaluator_r2.evaluate(lr_reg_predictions)
print(f"Linear Regression (Regression) - RMSE on Test Data: {lr_reg_rmse}")
print(f"Linear Regression (Regression) - R2 on Test Data: {lr_reg_r2}")

Fitting Linear Regression (Regression) CV model...


25/05/19 01:18:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Best Linear Regression (Regression) Hyperparameters:
{Param(parent='LinearRegression_e636edf94d7e', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 3, Param(parent='LinearRegression_e636edf94d7e', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LinearRegression_e636edf94d7e', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber'): 1.35, Param(parent='LinearRegression_e636edf94d7e', name='featuresCol', doc='features column name.'): 'features', Param(parent='LinearRegression_e636edf94d7e', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LinearRegression_e636edf94d7e', name='labelCol', doc='label column name.'): 'label', Param(parent='LinearRegression_e636edf94d7e', name='loss', doc='The loss function to be optimized. Supported

Predictions for LinearRegression_reg copied to local CSV: ../output/LinearRegression_reg_predictions.csv


Linear Regression (Regression) - RMSE on Test Data: 0.009737228580546839
Linear Regression (Regression) - R2 on Test Data: 0.9999999999996944


 Model 2.2: Gradient-Boosted Tree Regressor

In [46]:
gbt_reg = GBTRegressor(featuresCol="features", labelCol="label", seed=42)

gbt_reg_param_grid = ParamGridBuilder() \
    .addGrid(gbt_reg.maxIter, [10, 20]) \
    .addGrid(gbt_reg.maxDepth, [3, 5]) \
    .build()

gbt_reg_cv = CrossValidator(estimator=gbt_reg,
                            estimatorParamMaps=gbt_reg_param_grid,
                            evaluator=reg_evaluator_rmse, # Optimize for RMSE
                            numFolds=3, # Use 2 for faster run
                            parallelism=2)

print("Fitting GBT Regressor CV model...")
gbt_reg_cv_model = gbt_reg_cv.fit(train_data_reg)
best_gbt_reg_model = gbt_reg_cv_model.bestModel
print("Best GBT Regressor Hyperparameters:")
print(best_gbt_reg_model.extractParamMap())

gbt_reg_predictions = save_model_and_predictions(best_gbt_reg_model, "GBTRegressor", "reg", test_data_reg)

Fitting GBT Regressor CV model...


Best GBT Regressor Hyperparameters:
{Param(parent='GBTRegressor_28c91b6005f6', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False, Param(parent='GBTRegressor_28c91b6005f6', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10, Param(parent='GBTRegressor_28c91b6005f6', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classif

Predictions for GBTRegressor_reg copied to local CSV: ../output/GBTRegressor_reg_predictions.csv


In [47]:
gbt_reg_rmse = reg_evaluator_rmse.evaluate(gbt_reg_predictions)
gbt_reg_r2 = reg_evaluator_r2.evaluate(gbt_reg_predictions)
print(f"GBT Regressor - RMSE on Test Data: {gbt_reg_rmse}")
print(f"GBT Regressor - R2 on Test Data: {gbt_reg_r2}")

GBT Regressor - RMSE on Test Data: 11004.026788553107
GBT Regressor - R2 on Test Data: 0.6095892615385172


Compare Regression Models

In [48]:

reg_models_summary_data = [
    (str(best_lr_reg_model), lr_reg_rmse, lr_reg_r2),
    (str(best_gbt_reg_model), gbt_reg_rmse, gbt_reg_r2)
]
reg_summary_df = spark.createDataFrame(reg_models_summary_data, ["model", "RMSE", "R2"])
print("\nRegression Models Comparison:")
reg_summary_df.show(truncate=False)


Regression Models Comparison:
+------------------------------------------------------------------------------+--------------------+------------------+
|model                                                                         |RMSE                |R2                |
+------------------------------------------------------------------------------+--------------------+------------------+
|LinearRegressionModel: uid=LinearRegression_e636edf94d7e, numFeatures=87      |0.009737228580546839|0.9999999999996944|
|GBTRegressionModel: uid=GBTRegressor_28c91b6005f6, numTrees=20, numFeatures=87|11004.026788553107  |0.6095892615385172|
+------------------------------------------------------------------------------+--------------------+------------------+



In [49]:
reg_summary_path_hdfs = "project/output/regression_evaluation" # Directory for CSV parts
reg_summary_path_local_csv = "../output/regression_evaluation.csv"
reg_summary_df.coalesce(1) \
    .write.mode("overwrite").format("csv").option("sep", ",").option("header", "true") \
    .save(reg_summary_path_hdfs)
os.makedirs(os.path.dirname(reg_summary_path_local_csv), exist_ok=True)
run(f"hdfs dfs -cat {reg_summary_path_hdfs}/part*.csv > {reg_summary_path_local_csv}")
print(f"Regression evaluation summary saved to HDFS dir: {reg_summary_path_hdfs} and concatenated to local: {reg_summary_path_local_csv}")

train_data_reg.unpersist()
test_data_reg.unpersist()

print("\n--- All tasks completed. Stopping Spark session. ---")
spark.stop()

Regression evaluation summary saved to HDFS dir: project/output/regression_evaluation and concatenated to local: output/regression_evaluation.csv

--- All tasks completed. Stopping Spark session. ---
